# **Advanced Analysis with Folium - Clean Distance Markers**


This analysis helps to understand __*what is the distance of Equator, Railways, Highways, Coastlines and Cities*__ from launch sites more clearly.

Let's first import required Python packages for this lab:


In [1]:
!pip3 install folium
!pip3 install wget

In [1]:
import folium
import wget
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [3]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


#### Converting the dataframe to dictionary

In [5]:
launch_sites_dict = launch_sites_df.set_index('Launch Site').T.to_dict('list')
launch_sites_dict

{'CCAFS LC-40': [28.56230197, -80.57735648],
 'CCAFS SLC-40': [28.56319718, -80.57682003],
 'KSC LC-39A': [28.57325457, -80.64689529],
 'VAFB SLC-4E': [34.63283416, -120.6107455]}

# My Own Custom Functions for Markers

In [6]:
def add_marker_on_map(name, coordinate, circle_color='#d35400'):
    circle = folium.Circle(coordinate, radius=1000, color=circle_color, fill=True).add_child(folium.Popup(name))
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html="<div style='font-size: 12; color:{};'><b>{}</b></div>". format(circle_color,name)
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)
    
def add_text_distance_marker_on_map(name, coordinate, distance, ccolor='#4d0f1d'):
    txt="{}, {:0.2f} km".format(name, distance)
    folium.Marker(
    location=coordinate,
    icon=DivIcon(
            icon_size=(150,30),
            icon_anchor=(0,0),
            html='<div style="font-size: 9pt; font-weight:bold; color: {}">{}</div>'.format(ccolor, txt))
    ).add_to(site_map)

def add_type_marker_on_map(name, coordinate, ccolor='red', icon_type='train'):   
    marker = folium.map.Marker(
        location=coordinate, 
        popup=name,
        tooltip='click',
        icon=folium.Icon(color=ccolor, icon=icon_type, prefix='fa')
    )
    site_map.add_child(marker)
    
def add_type_distance_marker_on_map(name, coordinate, distance, ccolor='red', icon_type='bolt'):
    text = "{} {:0.2f}km".format(name, distance)
    marker = folium.map.Marker(
        location=coordinate, 
        popup=text,
        icon=folium.Icon(color=ccolor, icon=icon_type, prefix='fa')
    )
    site_map.add_child(marker)
    
def add_distance_marker_on_map(name, coordinate, distance, ccolor='#8D208B'):
    text = "{} {:0.2f}km".format(name, distance)
    marker = folium.map.Marker(
        location=coordinate, 
        tooltip=text,
        icon=plugins.BeautifyIcon(
            number=round(distance,0),
            border_color=ccolor,
            border_width=2,
            text_color=ccolor,
            inner_icon_style='margin-top:0px;')
    )
    site_map.add_child(marker)
    
def draw_distance_lines(place1, place2, lcolor='blue', icolor='red', tcolor='#4d0f1d'):
    add_type_marker_on_map(place2['name'], place2['coordinates'], ccolor=icolor, icon_type=place2['placetype'])
    points=[]
    points.append(place1['coordinates'])
    points.append(place2['coordinates'])
    distance = calculate_distance(place1['coordinates'][0], place1['coordinates'][1], place2['coordinates'][0], place2['coordinates'][1])
    add_text_distance_marker_on_map(place2['name'], place2['coordinates'], distance, tcolor)
    linetext = "{}{:0.2f} km".format(place2['name'], distance)
    # folium.plugins.AntPath(points, color=lcolor, weight=1.5, delay = .5, popup=linetext).add_to(site_map)
    folium.PolyLine(points, color=lcolor, weight=1.5, popup=linetext).add_to(site_map)
    
def draw_equater_line():
    points=[]
    points.append([0,195])
    points.append([0,-450])
    folium.plugins.AntPath(points, color='green', weight=1.5, delay = .5).add_to(site_map) 
    
def add_marker_cluster_on_map(name, coordinate, color):
    folium.Marker(
        location=coordinate,
        popup=name,
        icon=folium.Icon(color='white', icon_color=color),
    ).add_to(marker_cluster)

You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [7]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [8]:
# Initial the map
# site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# Zooming on a launch site 'VAFB SLC-4E' rather than NASA for better visibiliy of work mentioned in below tasks.
VAFB_SLC_4E = [34.632834, -120.610746]
site_map = folium.Map(location=VAFB_SLC_4E, zoom_start=3)
# CCAFS_SLC_40 = [28.563197177407144, -80.57682003124195]
# site_map = folium.Map(location=CCAFS_SLC_40, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for name,coordinate in launch_sites_dict.items():
    add_type_marker_on_map(name, coordinate,'red','rocket')
site_map

*TODO:* Mark down a point on the closest railway using MousePosition and calculate the distance between the railway point to the launch site.


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, coastline, highway, etc.


In [9]:
launchsite_VAFB = {'name':'VAFB SLC-4E', 'coordinates':[34.632834, -120.610746], 'placetype':'rocket'}
launchsite_KSC = {'name':'KSC LC-39A', 'coordinates':[28.573255, -80.646895], 'placetype':'rocket'}
launchsite_CCAFS_SLC = {'name':'CCAFS SLC-40', 'coordinates':[28.563197, -80.576820], 'placetype':'rocket'}


In [10]:
# Equator Line Distance
draw_equater_line()
equator1 = {'name':'Equator Distance from VAFB SLC-4E', 'coordinates':[0, -120.61], 'placetype':'map'}
draw_distance_lines(launchsite_VAFB, equator1,'green', 'green', '#325931')

equator2 = {'name':'Equator Distance from KSC LC-39A', 'coordinates':[0, -80.64651], 'placetype':'map'}
draw_distance_lines(launchsite_KSC, equator2, 'green', 'green', '#325931')

site_map

In [11]:
launchsite_VAFB_railway = {'name':'Railway from VAFB SLC-4E', 'coordinates':[34.63141, -120.62568], 'placetype':'train'}
launchsite_VAFB_highway = {'name':'Highway from VAFB SLC-4E', 'coordinates':[34.66992, -120.45753], 'placetype':'road'}
launchsite_VAFB_coastline = {'name':'Coastline from VAFB SLC-4E', 'coordinates':[34.6336, -120.62606], 'placetype':'life-ring'}
launchsite_VAFB_city = {'name':'City from VAFB SLC-4E', 'coordinates':[34.63658, -120.4542], 'placetype':'building'}

launchsite_KSC_railway = {'name':'Railway from KSC LC-39A', 'coordinates':[28.573255, -80.65411], 'placetype':'train'}
launchsite_KSC_highway = {'name':'Highway from KSC LC-39A', 'coordinates':[28.52594, -80.66357], 'placetype':'road'}
launchsite_KSC_coastline = {'name':'Coastline from KSC LC-39A', 'coordinates':[28.573255, -80.60669], 'placetype':'life-ring'}
launchsite_KSC_city = {'name':'City from KSC LC-39A', 'coordinates':[28.66288, -81.35925], 'placetype':'building'}

launchsite_CCAFS_SLC_railway = {'name':'Railway from CCAFS SLC-40', 'coordinates':[28.57367, -80.58472], 'placetype':'train'}
launchsite_CCAFS_SLC_highway = {'name':'Highway from CCAFS SLC-40', 'coordinates':[28.52557, -80.6347], 'placetype':'road'}
launchsite_CCAFS_SLC_coastline = {'name':'Coastline from CCAFS SLC-40', 'coordinates':[28.563197, -80.56772], 'placetype':'life-ring'}
launchsite_CCAFS_SLC_city = {'name':'City from CCAFS SLC-40', 'coordinates':[28.53748, -81.38672], 'placetype':'building'}



# Railway distances
site_map = folium.Map(location=VAFB_SLC_4E, zoom_start=3)

for name,coordinate in launch_sites_dict.items():
    add_type_marker_on_map(name, coordinate,'red','rocket')
    
draw_distance_lines(launchsite_VAFB, launchsite_VAFB_railway, 'darkgreen', 'green', '#325931')
draw_distance_lines(launchsite_KSC, launchsite_KSC_railway, 'darkgreen', 'green', '#325931')
draw_distance_lines(launchsite_CCAFS_SLC, launchsite_CCAFS_SLC_railway, 'darkgreen', 'green', '#325931')

site_map

In [12]:
# Highway distances
site_map = folium.Map(location=VAFB_SLC_4E, zoom_start=3)

for name,coordinate in launch_sites_dict.items():
    add_type_marker_on_map(name, coordinate,'red','rocket')

draw_distance_lines(launchsite_VAFB, launchsite_VAFB_highway,'brown', 'darkred', '#823604')
draw_distance_lines(launchsite_KSC, launchsite_KSC_highway, 'brown', 'darkred', '#823604')
draw_distance_lines(launchsite_CCAFS_SLC, launchsite_CCAFS_SLC_highway, 'brown', 'darkred', '#823604')

site_map

In [13]:
# Coastline distances
site_map = folium.Map(location=VAFB_SLC_4E, zoom_start=3)

for name,coordinate in launch_sites_dict.items():
    add_type_marker_on_map(name, coordinate,'red','rocket')
    
draw_distance_lines(launchsite_VAFB, launchsite_VAFB_coastline,'darkblue', 'darkblue', '#00237d')
draw_distance_lines(launchsite_KSC, launchsite_KSC_coastline, 'darkblue', 'darkblue', '#00237d')
draw_distance_lines(launchsite_CCAFS_SLC, launchsite_CCAFS_SLC_coastline, 'darkblue', 'darkblue', '#00237d')

site_map

In [14]:
# City distances
site_map = folium.Map(location=VAFB_SLC_4E, zoom_start=3)

for name,coordinate in launch_sites_dict.items():
    add_type_marker_on_map(name, coordinate,'red','rocket')
    
draw_distance_lines(launchsite_VAFB, launchsite_VAFB_city,'gray', 'gray', '#2e2e2e')
draw_distance_lines(launchsite_KSC, launchsite_KSC_city, 'gray', 'gray', '#2e2e2e')
draw_distance_lines(launchsite_CCAFS_SLC, launchsite_CCAFS_SLC_city, 'gray', 'gray', '#2e2e2e')

site_map